<a href="https://colab.research.google.com/github/moonlight-t1/42Curcus/blob/master/Machine_Learning/Perfect_Guide/04-3_Voting_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블 학습 - 보팅 분류기(Voting Classifier)

- 사이킷런은 보팅 방식의 앙상블을 구현한 `VotingClassifier` 클래스를 제공하고 있다.
- 위스콘신 유방암 데이터 세트는 유방암의 악성종양, 양성종양 여부를 결정하는 이진 분류 데이터 세트이며 종양의 크기, 모양 등의 형태와 관련된 많은 피처를 가지고 가지고 있다.
- 사이킷런은 `load_data_cancer()` 함수를 통해 자체에서 위스콘신 유방암 데이터 세트를 생성할 수 있다.

In [ ]:
# 위스콘신 유방암 데이터 로드
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

In [ ]:
# 로지스틱 회귀와 KNN을 기반으로 하여 소프트 보팅 방식으로 새롭게 보팅 분류기를 만든다

# 개별 모델은 로지스틱 회귀와 KNN 임. 
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=8)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 
vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)

# VotingClassifier 학습/예측/평가. 
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

- 사이킷런은 `VotingClassifier` 클래스를 이용해 보팅 분류기를 생성할 수 있다.
- `VotingClassifier` 클래스는 주요 생성 인자로 `estimator`와 `voting` 값을 입력 받는다. 
- `estimators`는 리스트 값으로 보팅에 사용될 여러 개의 `Classifier` 객체들을 튜플 형식으로 입력 받으며 `voting`은 `hard`시 하드 보팅, `soft`시 소프트 방식을 적용한다. (기본 `hard`)

In [ ]:
# 개별 모델의 학습/예측/평가.
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))

- 보팅 분류기가 정확도가 조금 높게 나타났는데, 보팅으로 여러 개의 기반 분류기를 결합한다고 해서 무조건 기반 분류기보다 예측 성능이 향상되지는 않는다. 데이터의 특성과 분포 등 다양한 요건에 따라 오히려 기반 분류기 중 가장 좋은 분류기의 성능이 보팅했을 떄보다 나을 수 있다.
- 그럼에도 불구하고 보팅을 포함해 배깅과 부스팅의 앙상블 방법은 전반적으로 단일 ML 알고리즘보다 뛰어난 예측 성능을 가지는 경우가 많다.
- ML 모델의 성능은 다양한 테스트 데이터에 의해 검증이 되므로 어떻게 높은 유연선을 가지고 현실에 대처할 수 있는가가 중요한 ML 모델의 평가요소가 된다. 이런 관점에서 편향-분산 트레이드 오프는 ML 모델이 극복해야 할 중요한 과제이다.


- 앙상블 학습에서는 결정 트리의 알고리즘의 단점을 수십 ~ 수천 개의 매우 많은 분류기를 결합해 다양한 상황을 학습하게 함으로써 극복하고 있다.
- 결정 트리 알고리즘의 장점은 그대로 취하고 단점은 보완하면서 편향-분산 트레이드오프의 효과를 극대화할 수 있다는 것이다.